In [1]:
import os
from pinecone import Pinecone
import uuid
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv



c:\Users\pratik p kakade\Desktop\infosys springboard\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [3]:
pc=Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("infosys-sprinboard") 


embed_model = SentenceTransformer('BAAI/bge-large-en-v1.5')


Loading weights: 100%|██████████| 391/391 [00:01<00:00, 325.06it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: BAAI/bge-large-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [4]:
policies = [
    # Category 1: Compliance & Legal
    {"category": "Compliance", "text": "Agents must explicitly state 'This call is being recorded for quality and training purposes' within the first 30 seconds of the interaction."},
    {"category": "Compliance", "text": "Before discussing account details, agents must verify at least two pieces of PII (e.g., full name and last four digits of SSN or birthdate)."},
    {"category": "Compliance", "text": "Agents must never read aloud or repeat back sensitive data like full credit card numbers, CVV codes, or passwords during the call."},
    {"category": "Compliance", "text": "If a customer requests to be removed from the calling list, agents must acknowledge the request immediately and confirm that the number will be added to the internal DNC registry."},
    {"category": "Compliance", "text": "Agents must ensure they are speaking directly with the patient or an authorized representative before disclosing any medical history or treatment information."},
    
    # Category 2: Empathy & Soft Skills
    {"category": "Empathy", "text": "Agents must use verbal acknowledgments (e.g., 'I see,' 'Understood') every 30–60 seconds to demonstrate they are actively processing the customer's information."},
    {"category": "Empathy", "text": "When a customer expresses frustration, agents must use a validation statement (e.g., 'I understand why that would be frustrating') before offering a technical solution."},
    {"category": "Empathy", "text": "Agents must take personal ownership of the issue using 'I' statements, such as 'I will personally look into this for you,' rather than blaming other departments."},
    {"category": "Empathy", "text": "Agents must allow the customer to finish their sentence completely. Interrupting a customer results in a score deduction."},
    {"category": "Empathy", "text": "Apologies must be specific to the issue (e.g., 'I’m sorry for the delay in your shipping') rather than a generic, robotic 'I apologize for the inconvenience.'"},
    
    # Category 3: Professionalism & Effectiveness
    {"category": "Professionalism", "text": "Calls must be opened with the standard greeting: 'Thank you for calling [Company Name], my name is [Agent Name]. How may I assist you today?'"},
    {"category": "Professionalism", "text": "Before placing a customer on hold, agents must ask for permission, state the reason for the hold, and provide an estimated wait time."},
    {"category": "Professionalism", "text": "Agents should avoid technical internal jargon or acronyms. Explanations must be in plain, accessible English."},
    {"category": "Professionalism", "text": "Agents must ask, 'Have I fully addressed all of your concerns today?' before ending the call to ensure the primary issue is resolved."},
    {"category": "Professionalism", "text": "Every call must conclude with a polite sign-off: 'Thank you for choosing [Company Name]. Have a wonderful rest of your day.'"}
]

In [8]:
vectors=[]

def upsert_policies():
    print(f"Uploading {len(policies)} policies in Pincone")

    for i,p in enumerate(policies):
        policy_vector=embed_model.encode(p["text"]).tolist()

        id=f"pol_{i}_{str(uuid.uuid4())[:8]}"

        vectors.append({
            "id":id,
            "values":policy_vector,
            "metadata":{
                "text":p["text"],
                "category":p["category"]
            }
        })

    print("Upserting policies")
    index.upsert(vectors=vectors)
    print("Uploaded Sucessfully")





In [9]:
upsert_policies()

Uploading 15 policies in Pincone
Upserting policies
Uploaded Sucessfully


In [12]:
vectors[0]["metadata"]

{'text': "Agents must explicitly state 'This call is being recorded for quality and training purposes' within the first 30 seconds of the interaction.",
 'category': 'Compliance'}

In [13]:
import mysql.connector

In [14]:
db_config={
    "host": "localhost",
    "user": "root",        
    "password": "MYSQL", 
    "database": "quality_auditor"
}

In [16]:
query="""Hello, I'm Sarah Miller. I'm calling to inquire about the AC-7892 air conditioner unit. I saw it on your website, and I had a few questions. First, what's the BTU rating? And second, does it come with a remote control, or is that sold separately?
"""

In [17]:
query_vector=embed_model.encode(query).tolist()

In [41]:
index.query(vector=query_vector,top_k=2,include_metadata=True)['matches'][1]['metadata']['category']


'Empathy'